#CHAPTER 10: Data Aggregation and Group Operations

##10.1 How to Think About Group Operations

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],
                                      dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [4]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [5]:
grouped.mean()

,data1
key1,
a,0.555881
b,0.705025


In [6]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -0.204708
      2       0.478943
b     1       1.965781
      2      -0.555730
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,1,2
key1,,
a,-0.204708,0.478943
b,1.965781,-0.555730


In [8]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]
df["data1"].groupby([states, years]).mean()

CA  2005    0.936175
    2006   -0.519439
OH  2005   -0.380219
    2006    1.029344
Name: data1, dtype: float64

In [9]:
df.groupby("key1").mean()
df.groupby("key2").mean(numeric_only=True)
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.204708  0.281746
     2     0.478943  0.769023
b    1     1.965781 -1.296221
     2    -0.555730  1.007189

In [10]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [11]:
df.groupby("key1", dropna=False).size()
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [12]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


###Iterating over Groups

In [13]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)


a
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
1    a     2  0.478943  0.769023
5    a  <NA>  1.393406  0.274992
b
  key1  key2     data1     data2
3    b     2 -0.555730  1.007189
4    b     1  1.965781 -1.296221


In [14]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)


('a', np.int64(1))
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
('a', np.int64(2))
  key1  key2     data1     data2
1    a     2  0.478943  0.769023
('b', np.int64(1))
  key1  key2     data1     data2
4    b     1  1.965781 -1.296221
('b', np.int64(2))
  key1  key2    data1     data2
3    b     2 -0.55573  1.007189


In [15]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221


In [16]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")

/tmp/ipython-input-1884927282.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


In [17]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)


data
      data1     data2
0 -0.204708  0.281746
1  0.478943  0.769023
2 -0.519439  1.246435
3 -0.555730  1.007189
4  1.965781 -1.296221
5  1.393406  0.274992
6  0.092908  0.228913
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


###Selecting a Column or Subset of Columns

In [18]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.281746
     2     0.769023
b    1    -1.296221
     2     1.007189

In [19]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped
s_grouped.mean()

key1  key2
a     1       0.281746
      2       0.769023
b     1      -1.296221
      2       1.007189
Name: data2, dtype: float64

###Grouping with Dictionaries and Series

In [20]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,1.352917,0.886429,-2.001637,-0.371843,1.669025
Steve,-0.438570,-0.539741,0.476985,3.248944,-1.021228
Wanda,-0.577087,NaN,NaN,0.523772,0.000940
Jill,1.343810,-0.713544,-0.831154,-2.370232,-1.860761
Trey,-0.860757,0.560145,-1.265934,0.119827,-1.063512


In [21]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

In [22]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

/tmp/ipython-input-2628240624.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


,blue,red
Joe,-2.373480,3.908371
Steve,3.725929,-1.999539
Wanda,0.523772,-0.576147
Jill,-3.201385,-1.230495
Trey,-1.146107,-1.364125


In [23]:
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis="columns").count()

/tmp/ipython-input-1085982663.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


###Grouping with Functions

In [24]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.352917,0.886429,-2.001637,-0.371843,1.669025
4,0.483052,-0.153399,-2.097088,-2.250405,-2.924273
5,-1.015657,-0.539741,0.476985,3.772716,-1.020287


###Grouping by Index Levels

In [25]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,1.352917,0.886429,-2.001637,-0.371843,1.669025
4,two,-0.860757,-0.713544,-1.265934,-2.370232,-1.860761
5,one,-0.577087,-0.539741,0.476985,0.523772,-1.021228


In [26]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                    [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.332883 -2.359419 -0.199543 -1.541996 -0.970736
1     -1.307030  0.286350  0.377984 -0.753887  0.331286
2      1.349742  0.069877  0.246674 -0.011862  1.004812
3      1.327195 -0.919262 -1.549106  0.022185  0.758363

In [27]:
hier_df.groupby(level="cty", axis="columns").count()

/tmp/ipython-input-2795167867.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


##10.2 Data Aggregation

In [28]:
df
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     0   -0.204708
      1    0.478943
b     3   -0.555730
      4    1.965781
Name: data1, dtype: float64

In [29]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.598113,0.494031
b,1,2.521511,2.303410


In [30]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.555881  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.705025  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.936175  1.393406   3.0  0.441920  0.283299  0.274992  0.278369   
b     1.335403  1.965781   2.0 -0.144516  1.628757 -1.296221 -0.720368   

                                    
           50%       75%       max  
key1                                
a     0.281746  0.525384  0.769023  
b    -0.144516  0.431337  1.007189  

[2 rows x 24 columns]

###Column-Wise and Multiple Function Application

In [ ]:
tips = pd.read_csv("examples/tips.csv")
tips.head()

In [ ]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

In [ ]:
grouped = tips.groupby(["day", "smoker"])

In [ ]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

In [35]:
grouped_pct.agg(["mean", "std", peak_to_peak])

NameError: name 'grouped_pct' is not defined

In [34]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

NameError: name 'grouped_pct' is not defined

In [33]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

KeyError: "Columns not found: 'tip_pct', 'total_bill'"

In [ ]:
result["tip_pct"]

In [ ]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

In [ ]:
grouped.agg({"tip" : np.max, "size" : "sum"})
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
             "size" : "sum"})

###Returning Aggregated Data Without Row Indexes

In [ ]:
grouped = tips.groupby(["day", "smoker"], as_index=False)
grouped.mean(numeric_only=True)

##10.3 Apply: General split-apply-combine

In [ ]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

In [ ]:
tips.groupby("smoker").apply(top)

In [ ]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

In [38]:
result = tips.groupby("smoker")["tip_pct"].describe()
result
result.unstack("smoker")

NameError: name 'tips' is not defined

###Suppressing the Group Keys

In [37]:
tips.groupby("smoker", group_keys=False).apply(top)

NameError: name 'tips' is not defined

###Quantile and Bucket Analysis

In [36]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

,data1
0,"(-1.23, 0.489]"
1,"(0.489, 2.208]"
2,"(-1.23, 0.489]"
3,"(-1.23, 0.489]"
4,"(0.489, 2.208]"
5,"(0.489, 2.208]"
6,"(-1.23, 0.489]"
7,"(-1.23, 0.489]"
8,"(-2.956, -1.23]"
9,"(-1.23, 0.489]"


In [39]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

/tmp/ipython-input-2015905501.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(quartiles)


min       max  count      mean
data1                                                     
(-2.956, -1.23] data1 -2.949343 -1.230179     94 -1.658818
                data2 -3.399312  1.670835     94 -0.033333
(-1.23, 0.489]  data1 -1.228918  0.488675    598 -0.329524
                data2 -2.989741  3.260383    598 -0.002622
(0.489, 2.208]  data1  0.489965  2.200997    298  1.065727
                data2 -3.745356  2.954439    298  0.078249
(2.208, 3.928]  data1  2.212303  3.927528     10  2.644253
                data2 -1.929776  1.765640     10  0.024750

In [41]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2                  \
                      min       max count      mean       min       max count   
data1                                                                           
(-2.956, -1.23] -2.949343 -1.230179    94 -1.658818 -3.399312  1.670835    94   
(-1.23, 0.489]  -1.228918  0.488675   598 -0.329524 -2.989741  3.260383   598   
(0.489, 2.208]   0.489965  2.200997   298  1.065727 -3.745356  2.954439   298   
(2.208, 3.928]   2.212303  3.927528    10  2.644253 -1.929776  1.765640    10   

                           
                     mean  
data1                      
(-2.956, -1.23] -0.033333  
(-1.23, 0.489]  -0.002622  
(0.489, 2.208]   0.078249  
(2.208, 3.928]   0.024750

In [42]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -2.949343 -0.685484    250 -1.212173
      data2 -3.399312  2.628441    250 -0.027045
1     data1 -0.683066 -0.030280    250 -0.368334
      data2 -2.630247  3.260383    250 -0.027845
2     data1 -0.027734  0.618965    250  0.295812
      data2 -3.056990  2.458842    250  0.014450
3     data1  0.623587  3.927528    250  1.248875
      data2 -3.745356  2.954439    250  0.115899

###Example: Filling Missing Values with Group-Specific Values

In [43]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s
s.fillna(s.mean())

,0
0,-0.767366
1,0.227290
2,-0.767366
3,-2.153545
4,-0.767366
5,-0.375842


In [44]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

,0
Ohio,0.329939
New York,0.981994
Vermont,1.105913
Florida,-1.613716
Oregon,1.561587
Nevada,0.406510
California,0.359244
Idaho,-0.614436


In [45]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data
data.groupby(group_key).size()
data.groupby(group_key).count()
data.groupby(group_key).mean()

,0
East,-0.100594
West,0.960416


In [46]:
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio          0.329939
      New York      0.981994
      Vermont      -0.100594
      Florida      -1.613716
West  Oregon        1.561587
      Nevada        0.960416
      California    0.359244
      Idaho         0.960416
dtype: float64

In [47]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_func)

East  Ohio          0.329939
      New York      0.981994
      Vermont       0.500000
      Florida      -1.613716
West  Oregon        1.561587
      Nevada       -1.000000
      California    0.359244
      Idaho        -1.000000
dtype: float64

###Example: Random Sampling and Permutation

In [48]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [49]:
deck.head(13)

,0
AH,1
2H,2
3H,3
4H,4
5H,5
6H,6
7H,7
8H,8
9H,9
10H,10


In [50]:

def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

,0
4D,4
QH,10
8S,8
7D,7
9C,9


In [51]:
def get_suit(card):
    # last letter is suit
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  6C     6
   KC    10
D  7D     7
   3D     3
H  7H     7
   9H     9
S  2S     2
   QS    10
dtype: int64

In [52]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

,0
AC,1
3C,3
5D,5
4D,4
10H,10
7H,7
QS,10
7S,7


###Example: Group Weighted Average and Correlation

In [53]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})
df

,category,data,weights
0,a,-1.691656,0.955905
1,a,0.511622,0.012745
2,a,-0.401675,0.137009
3,a,0.968578,0.763037
4,b,-1.818215,0.492472
5,b,0.279963,0.832908
6,b,-0.200819,0.658331
7,b,-0.217221,0.612009


In [54]:
grouped = df.groupby("category")
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

grouped.apply(get_wavg)

/tmp/ipython-input-505683550.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(get_wavg)


,0
category,
a,-0.495807
b,-0.357273


In [55]:
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True,
                       index_col=0)
close_px.info()
close_px.tail(4)

FileNotFoundError: [Errno 2] No such file or directory: 'examples/stock_px.csv'

In [56]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [57]:
rets = close_px.pct_change().dropna()

NameError: name 'close_px' is not defined

In [58]:
def get_year(x):
    return x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

NameError: name 'rets' is not defined

In [ ]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])
by_year.apply(corr_aapl_msft)

###Example: Group-Wise Linear Regression

In [59]:
import statsmodels.api as sm
def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

##10.4 Group Transforms and “Unwrapped” GroupBys

In [60]:
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

NameError: name 'by_year' is not defined

In [61]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [62]:
g = df.groupby('key')['value']
g.mean()

,value
key,
a,4.5
b,5.5
c,6.5


In [63]:
def get_mean(group):
    return group.mean()
g.transform(get_mean)

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [64]:
g.transform('mean')

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [65]:
def times_two(group):
    return group * 2
g.transform(times_two)

,value
0,0.0
1,2.0
2,4.0
3,6.0
4,8.0
5,10.0
6,12.0
7,14.0
8,16.0
9,18.0


In [66]:
def get_ranks(group):
    return group.rank(ascending=False)
g.transform(get_ranks)

,value
0,4.0
1,4.0
2,4.0
3,3.0
4,3.0
5,3.0
6,2.0
7,2.0
8,2.0
9,1.0


In [67]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [68]:
g.transform(normalize)
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [69]:
g.transform('mean')
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

,value
0,-1.161895
1,-1.161895
2,-1.161895
3,-0.387298
4,-0.387298
5,-0.387298
6,0.387298
7,0.387298
8,0.387298
9,1.161895


##10.5 Pivot Tables and Cross-Tabulation

In [70]:
tips.head()
tips.pivot_table(index=["day", "smoker"],
                 values=["size", "tip", "tip_pct", "total_bill"])

NameError: name 'tips' is not defined

In [71]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

NameError: name 'tips' is not defined

In [ ]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], margins=True)

In [ ]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

In [ ]:
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=0)

###Cross-Tabulations: Crosstab

In [72]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")

<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-30910860.py:13: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_table(StringIO(data), sep="\s+")


In [73]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [74]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [75]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

NameError: name 'tips' is not defined

In [76]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

#Kết thúc
